In [7]:
import pandas as pd
import numpy as np
import promptlayer

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chat_models import PromptLayerChatOpenAI

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from rich.console import Console
from rich.table import Table
console = Console()

from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')
WOLFRAM_ALPHA_APPID = os.getenv('WOLFRAM_ALPHA_APPID')
PROMPTLAYER_API_KEY = os.getenv('PROMPTLAYER_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENV = os.getenv('PINECONE_ENV')



from memory import PineconeMemory
from memory import BrainMemoryCreator, BrainMemoryRetriever



# Initialize memory and make sure it is empty.
# this is particularly important for indexing and referencing pinecone memory

memory = PineconeMemory()


print('Using memory of type: ' + memory.__class__.__name__)


# Initialize the agent
openai = promptlayer.openai
chat = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY)


Using memory of type: PineconeMemory


In [8]:
memory.clear()
full_message_history = []

In [9]:
reviews = pd.read_csv('reviews.csv')
querry = {0: "What are the top traits that customers appreciate the most about a product? Provide a description of each trait. ",
          1: "What are the top traits that customers appreciate the least about a product? Provide a description of each trait. ",
          2: "What are the improvements that can be brought to this product? Provide a description of each trait. "}

SystemMessage = """Answer the question based on the context and reviews below. You will answer with bulletpoints and extra clarity as a profesional developer. If the question cannot be answered using the information provided answer with "I don't know".

Context: You are looking at a product sold on amazon.com. We are a competing product development team. Our scope is to better understand the clients need with our product in order to improve the product. """

system_message_prompt = SystemMessagePromptTemplate.from_template(SystemMessage)


HumanMessage = """Question:  {inputQuestion}
                Reviews: {inputReviews} """

human_message_prompt = HumanMessagePromptTemplate.from_template(HumanMessage)

AiMessage = """Answer: """
ai_message_prompt = AIMessagePromptTemplate.from_template(AiMessage)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt, ai_message_prompt])

In [6]:
# Pass the reviews and querry to the promptlayer, get the results and add them to the memory

if len(querry) > 0:
    for i in range(len(reviews)):
        for j in range(len(querry)):
            # PROMPTLAYER
            prompt = chat_prompt.format_prompt(
                inputQuestion=querry[j],
                inputReviews=reviews['review'][i]
            ).to_messages()
            results = chat(prompt) 
            print(results)
       
            full_memory_to_add = f" Human Message: {querry[j]} " \
                f"\n Assistant Reply: {results} " \
                f"\n Reviews: {reviews['review'][i]} "
            memory.add(full_memory_to_add)

            short_memory_to_add = f" Human Question: {querry[j]} " \
                f"\n Assistant Answer: {results} "
            memory.add(short_memory_to_add)

# template_dict = promptlayer.prompts.get("reviewsPromptFeatures_test1")
# https://python.langchain.com/en/latest/modules/models/chat/integrations/promptlayer_chatopenai.html


# memory.get_relevant("Assistant Reply", num_relevant=6)


memory.get_stats()

content='The top traits that customers appreciate the most about this product, based on the reviews, are:\n\n- Easy to use\n- Gems/stickers stay on hair and other surfaces for hours, even after brushing or showering\n- Makes a great gift for both children and adults\n- Fun to use for festivals, concerts, or special events\n- Versatile - can be used on hair, clothes, shoes, accessories, and more\n- Comes with a good amount of gems/stickers in various shapes and sizes\n- Can be reused on other surfaces\n- Adds a cute and fun pop of sparkle to any outfit or hairstyle.' additional_kwargs={}
content="Top traits that customers appreciate the least about a product: \n\n* Stickers are difficult to remove from hair and keep falling off easily.\n* Adhesive doesn't last long and does not stick to thicker fabrics, bedding, and couches.\n* The product is made of cheap plastic and doesn't work as expected, sometimes breaking after a few uses.\n* The product does not look like the picture or descript

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12},
                'Write a memory stream for a GPT based agent. This paper can be used as a refference https://arxiv.org/pdf/2304.03442.pdf': {'vector_count': 1},
                'Write a summary on how to use OpenAi in product development within 10 prompts, having the reviews of a product as input': {'vector_count': 1}},
 'total_vector_count': 14}

In [7]:
Knowns = memory.get_relevant("Assistant Answer", num_relevant=12)

In [8]:
# I need to split this message at "Reviews: " and only use what is until reviews.

def split_string(string, separator):
    return string.split(separator, 1)[0]

# QA should have all the data from knowns, but up to the reviews part

QA = []
for i in range(len(Knowns)):
    QA.append(split_string(Knowns[i], "Reviews: "))
QA

[" Human Message: What are the top traits that customers appreciate the most about a product? Provide a description of each trait.  \n Assistant Reply: content='Traits that customers appreciate about the product:\\n\\n- Easy to use\\n- Adhesive gems stay on hair/fabrics/books/etc for a long time\\n- Gems are easily removable\\n- Gems are sticky and less damaging to hair than staples\\n- Gems are cute, fun and add a pop to any outfit or hairstyle\\n- Large number of gems included in the package\\n- Gems can be reused on other things\\n- Gems come in different shapes, colors, and sizes\\n- Good gift for children, tweens, and adults\\n- Cute addition to the whole outfit look\\n- Perfect starter kit for youngsters that love pizzazz.\\n- Ethnically insensitive product naming\\n- The product is reasonable in price compared to other similar products.' additional_kwargs={} \n ",
 "Human Message: What are the top traits that customers appreciate the least about a product? Provide a description 

In [30]:
#I want to save QA locally
QA = pd.DataFrame(QA)
QA.to_csv('QA.csv')

In [10]:
# I want to load QA and as a list
QA = pd.read_csv('QA.csv')
QA = QA['0'].tolist()

### QA Will be the input data for my next call
### I will need a code that will use the QA as input and get the answer from the agent.
### The interaction with the Agent will be added to memory
### The agent can use a lot of tools, like wolfram, google, etc.
### I will need to add the tools to the agent
### The agent will need to be able to use the memory to get the answers
### The agent will need to be able to use the memory to get the questions that need to be answered
### The agent will use a product development template to answer the questions in order to improve the product

In [11]:
from langchain.agents import initialize_agent, load_tools, Tool
from langchain.llms import OpenAI

from langchain import SerpAPIWrapper, LLMChain, LLMMathChain, Wikipedia
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper

from langchain.chains.conversation.memory import ConversationBufferMemory

from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())

import json

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [15]:
conversationMemory = ConversationBufferMemory(memory_key="chat_history")
search = SerpAPIWrapper()
llm_math = LLMMathChain(llm=llm, verbose=True)
wolfram = WolframAlphaAPIWrapper()


tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math.run,
        description="useful for when you need to answer questions about Divide, Multiply, Add and Subtract"
    ),
    Tool(
        name="Wolfram",
        func=wolfram.run,
        description="Useful for when you need to answer questions about math, science, geography, history, politics, arts, business, finance, product materials, material science"
    ),
    Tool(
        name="SearchWiki",
        func=docstore.search,
        description="useful for when you need to ask Wikipedia, a general purpose encyclopedia,  with search"
    ),
    Tool(
        name="LookupWiki",
        func=docstore.lookup,
        description="useful for when you need to ask Wikipedia, a general purpose encyclopedia,  with lookup"
    )
]

In [22]:
QA

["Human Message: What are the improvements that can be brought to this product? Provide a description of each trait.  \nAssistant Reply: content='Based on the reviews, it is difficult to identify specific improvements for the product. However, the following observations can be made to help in product development:\\n- The adhesive should be improved so that the gems stick longer to surfaces\\n- The instructions on how to use the device can be improved, making it more user-friendly for anyone who wants to use it\\n- The product descriptions and images need to accurately reflect the product so that there is no confusion when it arrives\\n- The product should be tested to ensure proper functionality and durability before shipping\\n- More refills should be included in the kit to avoid continually buying additional ones' additional_kwargs={} \n",
 "Human Message: What are the improvements that can be brought to this product? Provide a description of each trait.  \nAssistant Reply: content='

In [53]:
QA = [
"Human Message: What are the improvements that can be brought to this product? Provide a description of each trait.  \nAssistant Reply: content='Based on the reviews, it is difficult to identify specific improvements for the product. However, the following observations can be made to help in product development:\\n- The adhesive should be improved so that the gems stick longer to surfaces\\n- The instructions on how to use the device can be improved, making it more user-friendly for anyone who wants to use it\\n- The product descriptions and images need to accurately reflect the product so that there is no confusion when it arrives\\n- The product should be tested to ensure proper functionality and durability before shipping\\n- More refills should be included in the kit to avoid continually buying additional ones' additional_kwargs={} \n",
"Human Message: What are the improvements that can be brought to this product? Provide a description of each trait.  \nAssistant Reply: content='Some potential improvements that can be made to this product based on the reviews are:\\n\\n- Increase the number of gems included or sell larger refill packs\\n- Improve the quality of the plastic components, as some reviewers mentioned that the product feels flimsy or breaks easily\\n- Increase the strength of the adhesive on the gems to ensure they stay in place longer\\n- Improve the design of the machine to prevent gems from getting stuck or falling out during use\\n- Provide clearer instructions or tutorials for how to use the product, especially for younger kids or adults who are not familiar with it\\n- Consider including additional accessories or features that would enhance the user experience, such as different gem shapes or sizes, or the ability to customize the machine with stickers or decorations.' additional_kwargs={} \n"
]


template = """
Assistant is designed to independently be able to assist with creating industrial products, with access to search, Wolfram Alpha, and a calculator. 
Assistent can also summarize information from reviews and generate a report outlining potential product improvements, from answering simple questions to providing 
in-depth explanations and discussions on a wide range of topics.

Your decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.

You have the ability to call external tools. When you call an external tool, it will provide you with some information, which you can use when answering.
Please remember: your answer should be as detailed as possible. If you are going to cite information from the internet, please add the corresponding URL at the appropriate location.

CONTRAINT:  No user assistance

PERFORMANCE EVALUATION:

1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities. 
2. Constructively self-criticize your big-picture behavior constantly.
3. Reflect on past decisions and strategies to refine your approach.
4. Every command has a cost, so be smart and efficient. Aim to complete tasks in the least number of steps.

You should only respond in JSON format as described below

RESPONSE FORMAT:
{
    "thoughts":
    {
        "text": "thought",
        "reasoning": "reasoning",
        "plan": "- short bulleted\n- list that conveys\n- long-term plan",
        "criticism": "constructive self-criticism",
        "speak": "thoughts summary to say to user"
    },
    "command": {
        "name": "command name",
        "args":{
            "arg name": "value"
        }
    }
}

Ensure the response can be parsed by Python json.loads

GOAL: Improve the product based on the reviews. Be very detailed with precise reasoning and a plan for improvement. All details required to implement must be provided.
Example: "potential improvements can include increasing the number of gems included" is not sufficient. You must provide a detailed number of gems and why that number of gems. If you do not know the answer, you must continue to search online for the information.


Historical Information:
"Human Message: What are the improvements that can be brought to this product? Provide a description of each trait.  \nAssistant Reply: content='Based on the reviews, it is difficult to identify specific improvements for the product. However, the following observations can be made to help in product development:\\n- The adhesive should be improved so that the gems stick longer to surfaces\\n- The instructions on how to use the device can be improved, making it more user-friendly for anyone who wants to use it\\n- The product descriptions and images need to accurately reflect the product so that there is no confusion when it arrives\\n- The product should be tested to ensure proper functionality and durability before shipping\\n- More refills should be included in the kit to avoid continually buying additional ones' additional_kwargs={} \n",
"Human Message: What are the improvements that can be brought to this product? Provide a description of each trait.  \nAssistant Reply: content='Some potential improvements that can be made to this product based on the reviews are:\\n\\n- Increase the number of gems included or sell larger refill packs\\n- Improve the quality of the plastic components, as some reviewers mentioned that the product feels flimsy or breaks easily\\n- Increase the strength of the adhesive on the gems to ensure they stay in place longer\\n- Improve the design of the machine to prevent gems from getting stuck or falling out during use\\n- Provide clearer instructions or tutorials for how to use the product, especially for younger kids or adults who are not familiar with it\\n- Consider including additional accessories or features that would enhance the user experience, such as different gem shapes or sizes, or the ability to customize the machine with stickers or decorations.' additional_kwargs={} \n"


Assistant:"""

In [54]:
agent_chain = initialize_agent(tools,
                               llm,
                               verbose=True,
                               conversationMemory=conversationMemory)

In [55]:
agent_chain

AgentExecutor(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x13e4e91c0>, verbose=True, agent=ZeroShotAgent(llm_chain=LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x13e4e91c0>, verbose=False, prompt=PromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nSearch: useful for when you need to answer questions about current events\nCalculator: useful for when you need to answer questions about Divide, Multiply, Add and Subtract\nWolfram: Useful for when you need to answer questions about math, science, geography, history, politics, arts, business, finance, product materials, material science\nSearchWiki: useful for when you need to ask Wikipedia, a general purpose encyclopedia,  with search\nLookupWiki: useful for when you need to ask Wikipedia, a

In [56]:
output = agent_chain.run(template)



> Entering new AgentExecutor chain...
 I need to review the reviews and analyze the product to identify potential improvements
Action: SearchWiki
Action Input: Reviews of the product
Observation: A review is an evaluation of a publication, product, service, or company or a critical take on current affairs in literature, politics or culture. In addition to a critical evaluation, the review's author may assign the work a rating to indicate its relative merit. 
A compilation of reviews may itself be called a review. 
Reviews can apply to a movie (a movie review), video game (video game review), musical composition (music review of a composition or recording), book (book review); a piece of hardware like a car, home appliance, or computer; or software such as business software, sales software; or an event or performance, such as a live music concert, play, musical theater show, dance show or art exhibition
In the cultural sphere, The New York Review of Books, for instance, is a collectio

In [57]:
output

'Potential improvements to the product can include increasing the number of gems included, improving the quality of the plastic components, increasing the strength of the adhesive on the gems, improving the design of the machine, providing clearer instructions or tutorials, and considering additional accessories or features that would enhance the user experience.'

In [100]:
while True:
    user_input = input("Human: ")
    if user_input == "exit":
        break
    response = agent_chain.run(input=user_input)
    print(response)



> Entering new AgentExecutor chain...
AI: Thought: Do I need to use a tool? No
AI: To improve the product detailed in the reviews, you can use a variety of methodologies such as user-centered design, lean manufacturing, and Six Sigma. Each of these methodologies has its own set of tools and techniques that can be used to improve the product.

> Finished chain.
To improve the product detailed in the reviews, you can use a variety of methodologies such as user-centered design, lean manufacturing, and Six Sigma. Each of these methodologies has its own set of tools and techniques that can be used to improve the product.


> Entering new AgentExecutor chain...

What are the benefits of using Six Sigma?

AI: The benefits of using Six Sigma include improved customer satisfaction, increased efficiency, reduced costs, improved quality, and increased profitability. Six Sigma also helps to identify and eliminate sources of variation and defects in processes, resulting in improved process perfor